In [1]:
import numpy as np
import pandas as pd
import gc
import time
import category_encoders as ce
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy.cluster.vq import kmeans2, whiten
from sklearn.preprocessing import Imputer
from sklearn.decomposition import truncated_svd
import category_encoders as ce
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn import preprocessing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

num_rows = None
EPS = 1e-100

/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
descretize = lambda x, n: list(map(str, list(pd.qcut(x, n, duplicates='drop'))))

def binary_encoder(df, n_train):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    enc = ce.BinaryEncoder(impute_missing=True, cols=categorical_columns).fit(df[0:n_train], df[0:n_train]['TARGET'])
    df = enc.transform(df)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df[new_columns]

def helmert_encoder(df, n_train):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    enc = ce.HelmertEncoder(impute_missing=True, cols=categorical_columns).fit(df[0:n_train], df[0:n_train]['TARGET'])
    df = enc.transform(df)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df[new_columns]

def target_encoder(df, n_train):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    enc = ce.TargetEncoder(impute_missing=True, cols=categorical_columns).fit(df[0:n_train], df[0:n_train]['TARGET'])
    df = enc.transform(df)
    return df[categorical_columns]

def poly_encoder(df, n_train):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    enc = ce.PolynomialEncoder(impute_missing=True, cols=categorical_columns).fit(df[0:n_train], df[0:n_train]['TARGET'])
    df = enc.transform(df)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df[new_columns]

In [120]:
test_file_path = "Level_1_stack_new/test_ann_a_1.csv"
validation_file_path = 'Level_1_stack_new/validation_ann_a_1.csv'
num_folds = 5

In [121]:
df = pd.read_csv('../data/SureshFeaturesAug16_2.csv', header=0, index_col=None)

In [122]:
gc.collect()

56

In [123]:
goran_features = pd.read_csv('../goran-data/goranm_feats_v3.csv', header=0, index_col=None)
del goran_features['SK_ID_CURR']
del goran_features['IS_TRAIN']
goran_features.head()


,AMT_GOODS_PRICE_TO_CREDIT,AMT_CREDIT_INSTALLMENTS,AMT_CREDIT_INTEREST_MAYBE,AMT_CREDIT_INTEREST_MAYBE_RAT,AMT_INCOME_TOTAL_RAT_CREDIT,NAME_CONTRACT_TYPE_Consumer_loans,NAME_CONTRACT_TYPE_Cash_loans,NAME_CONTRACT_TYPE_Revolving_loans,NAME_CONTRACT_TYPE_XNA,PRE_RECORD_COUNT,...,STCK_PAY_720L_.,STCK_PAY_180L_.,STCK_PAY_480_.,STCK_CC_6_.,STCK_BERBAL_6_.,EXT_MEAN,TERM_BEFORE_END,DAYS_DECISION_MEAN,AMT_CREDIT_LIMIT_ACTUAL_6,AMT_CREDIT_LIMIT_ACTUAL_12
0,0.863262,16.461104,11389.5,0.028012,0.144444,0.0,1.0,0.0,0.0,1.0,...,0.080220,0.076729,0.080736,NaN,0.160535,0.161787,1.0,-606.0,NaN,NaN
1,0.873211,36.234085,8356.5,0.006460,0.348611,1.0,2.0,0.0,0.0,3.0,...,0.053463,NaN,NaN,NaN,NaN,NaN,3.0,-3915.0,NaN,NaN
2,1.000000,20.000000,0.0,0.000000,0.166667,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-815.0,NaN,NaN
3,0.949845,10.532818,15817.5,0.050586,0.183333,5.0,2.0,2.0,0.0,9.0,...,0.081061,0.069552,0.073751,0.03353,NaN,NaN,2.0,-2452.0,270000.0,270000.0
4,1.000000,23.461618,10093.5,0.019675,0.351852,4.0,2.0,0.0,0.0,6.0,...,0.074988,0.090350,0.076197,NaN,NaN,NaN,4.0,-7337.0,NaN,NaN


In [124]:
uniques = [f for f in goran_features.columns if f not in df.columns]

In [125]:
df = pd.concat([df, goran_features[uniques]], axis=1)

In [126]:
new_features2 = pd.read_csv('../data/selected_features2.csv', header=0, index_col=None)

In [127]:
uniques = [f for f in new_features2.columns if f not in df.columns]

In [128]:
df = pd.concat([df, new_features2[uniques]], axis=1)

In [129]:
df.shape

(356255, 816)

In [130]:
from sklearn.externals import joblib

shubin_df = joblib.load('../data/stacked_featureSet')


In [131]:
shubin_df.head()

,SK_ID_CURR,solution1_pred_lgb,solution2_pred_lgb,bureau_v2_solution,bureau_solution2,application_solution2,pos_v1_solution2,prev_application_solution2,EXT1,EXT2
0,100002,0.218051,0.264317,0.105383,0.092550,0.438352,0.079057,0.086284,0.234845,0.230231
1,100003,0.012417,0.018419,0.025442,0.028613,0.047676,0.061434,0.017387,0.013653,0.019212
2,100004,0.036228,0.039559,0.041481,0.055773,0.043101,0.077291,0.119400,0.042936,0.036041
3,100006,0.040368,0.047467,0.101213,0.101191,0.045045,0.092093,0.083527,0.036698,0.040683
4,100007,0.051251,0.044134,0.048242,0.056467,0.095363,0.039969,0.078971,0.046074,0.056351


In [132]:
uniques = [f for f in shubin_df.columns if f not in df.columns]

In [133]:
df = pd.concat([df, shubin_df[uniques]], axis=1)

In [134]:
df.shape

(356255, 825)

In [135]:
train = pd.read_csv('../data/application_train.csv.zip', nrows=num_rows)
n_train = train.shape[0]
test = pd.read_csv('../data/application_test.csv.zip', nrows=num_rows)
    
new_df = pd.concat([train, test], axis=0)
gc.collect()

56

In [136]:
categorical_columns = [col for col in train.columns if train[col].dtype == 'object']

In [137]:
encoder = preprocessing.LabelEncoder()

for f in categorical_columns:
    if new_df[f].dtype == 'object':
        new_df[f] = encoder.fit_transform(new_df[f].apply(str).values)

In [138]:
def do_countuniq( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Counting unqiue ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_mean(df, group_cols, counted, agg_name, agg_type='float32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Calculating mean of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_count(df, group_cols, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Aggregating by ", group_cols , '...' )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

In [139]:
counts_columns = []
for f_0 in categorical_columns:
    for f_1 in [x for x in categorical_columns if x != f_0] :
        new_df = do_countuniq(new_df, [f_0], f_1,
                      f_0 + '-' + f_1 + '_cunique', 'uint16', show_max=True); gc.collect()
        counts_columns.append(f_0 + '-' + f_1 + '_cunique')

Counting unqiue  CODE_GENDER  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-CODE_GENDER_cunique max value =  3
Counting unqiue  FLAG_OWN_CAR  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-FLAG_OWN_CAR_cunique max value =  2
Counting unqiue  FLAG_OWN_REALTY  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-FLAG_OWN_REALTY_cunique max value =  2
Counting unqiue  NAME_TYPE_SUITE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_TYPE_SUITE_cunique max value =  8
Counting unqiue  NAME_INCOME_TYPE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_INCOME_TYPE_cunique max value =  8
Counting unqiue  NAME_EDUCATION_TYPE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_EDUCATION_TYPE_cunique max value =  5
Counting unqiue  NAME_FAMILY_STATUS  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_FAMILY_STATUS_cunique max value =  6
Counting unqiue  NAME_HOUSING_TYPE  by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE-NAME_HOUSING_TYPE_cunique max value =  6
Counting u

NAME_TYPE_SUITE-WEEKDAY_APPR_PROCESS_START_cunique max value =  7
Counting unqiue  ORGANIZATION_TYPE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-ORGANIZATION_TYPE_cunique max value =  58
Counting unqiue  FONDKAPREMONT_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-FONDKAPREMONT_MODE_cunique max value =  5
Counting unqiue  HOUSETYPE_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-HOUSETYPE_MODE_cunique max value =  4
Counting unqiue  WALLSMATERIAL_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-WALLSMATERIAL_MODE_cunique max value =  8
Counting unqiue  EMERGENCYSTATE_MODE  by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE-EMERGENCYSTATE_MODE_cunique max value =  3
Counting unqiue  NAME_CONTRACT_TYPE  by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE-NAME_CONTRACT_TYPE_cunique max value =  2
Counting unqiue  CODE_GENDER  by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE-CODE_GENDER_cunique max value =  3
Counting unqiue  FLAG_OWN_CAR  by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE-FLAG_OWN_CAR_

OCCUPATION_TYPE-NAME_CONTRACT_TYPE_cunique max value =  2
Counting unqiue  CODE_GENDER  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-CODE_GENDER_cunique max value =  3
Counting unqiue  FLAG_OWN_CAR  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-FLAG_OWN_CAR_cunique max value =  2
Counting unqiue  FLAG_OWN_REALTY  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-FLAG_OWN_REALTY_cunique max value =  2
Counting unqiue  NAME_TYPE_SUITE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_TYPE_SUITE_cunique max value =  8
Counting unqiue  NAME_INCOME_TYPE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_INCOME_TYPE_cunique max value =  8
Counting unqiue  NAME_EDUCATION_TYPE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_EDUCATION_TYPE_cunique max value =  5
Counting unqiue  NAME_FAMILY_STATUS  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_FAMILY_STATUS_cunique max value =  6
Counting unqiue  NAME_HOUSING_TYPE  by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE-NAME_HOUSING_TYPE_cunique max value =  6


HOUSETYPE_MODE-NAME_INCOME_TYPE_cunique max value =  8
Counting unqiue  NAME_EDUCATION_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-NAME_EDUCATION_TYPE_cunique max value =  5
Counting unqiue  NAME_FAMILY_STATUS  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-NAME_FAMILY_STATUS_cunique max value =  6
Counting unqiue  NAME_HOUSING_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-NAME_HOUSING_TYPE_cunique max value =  6
Counting unqiue  OCCUPATION_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-OCCUPATION_TYPE_cunique max value =  19
Counting unqiue  WEEKDAY_APPR_PROCESS_START  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-WEEKDAY_APPR_PROCESS_START_cunique max value =  7
Counting unqiue  ORGANIZATION_TYPE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-ORGANIZATION_TYPE_cunique max value =  58
Counting unqiue  FONDKAPREMONT_MODE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-FONDKAPREMONT_MODE_cunique max value =  5
Counting unqiue  WALLSMATERIAL_MODE  by  ['HOUSETYPE_MODE'] ...
HOUSETYPE_MODE-WALLSMATE

In [140]:
count_columns = []
for f_0 in categorical_columns:
        new_df = do_count(new_df, [f_0],
                      f_0  + '_count', 'uint16', show_max=True); gc.collect()
        count_columns.append(f_0  + '_count')

Aggregating by  ['NAME_CONTRACT_TYPE'] ...
NAME_CONTRACT_TYPE_count max value =  326537
Aggregating by  ['CODE_GENDER'] ...
CODE_GENDER_count max value =  235126
Aggregating by  ['FLAG_OWN_CAR'] ...
FLAG_OWN_CAR_count max value =  235235
Aggregating by  ['FLAG_OWN_REALTY'] ...
FLAG_OWN_REALTY_count max value =  246970
Aggregating by  ['NAME_TYPE_SUITE'] ...
NAME_TYPE_SUITE_count max value =  288253
Aggregating by  ['NAME_INCOME_TYPE'] ...
NAME_INCOME_TYPE_count max value =  183307
Aggregating by  ['NAME_EDUCATION_TYPE'] ...
NAME_EDUCATION_TYPE_count max value =  252379
Aggregating by  ['NAME_FAMILY_STATUS'] ...
NAME_FAMILY_STATUS_count max value =  228715
Aggregating by  ['NAME_HOUSING_TYPE'] ...
NAME_HOUSING_TYPE_count max value =  316513
Aggregating by  ['OCCUPATION_TYPE'] ...
OCCUPATION_TYPE_count max value =  111996
Aggregating by  ['WEEKDAY_APPR_PROCESS_START'] ...
WEEKDAY_APPR_PROCESS_START_count max value =  63652
Aggregating by  ['ORGANIZATION_TYPE'] ...
ORGANIZATION_TYPE_count

In [141]:
feats = [f for f in new_df.columns if f not in df.columns]

In [142]:
df = pd.concat([df, new_df[feats]], axis=1)

In [143]:
gc.collect()

7

In [144]:
df.shape

(356255, 1202)

In [145]:
cols_to_drop = [

  "FLAG_DOCUMENT_2",

  "FLAG_DOCUMENT_7",

  "FLAG_DOCUMENT_10",

  "FLAG_DOCUMENT_12",

  "FLAG_DOCUMENT_13",

  "FLAG_DOCUMENT_14",

  "FLAG_DOCUMENT_15",

  "FLAG_DOCUMENT_16",

  "FLAG_DOCUMENT_17",

  "FLAG_DOCUMENT_18",

  "FLAG_DOCUMENT_19",

  "FLAG_DOCUMENT_20",

  "FLAG_DOCUMENT_21",

  "PREV_NAME_CONTRACT_TYPE_Consumer_loans",

  "PREV_NAME_CONTRACT_TYPE_XNA",

  "PB_CNT_NAME_CONTRACT_STATUS_Amortized_debt",

  "MAX_DATA_ALL",

  "MIN_DATA_ALL",

  "MAX_MIN_DURATION",

  "MAX_AMT_CREDIT_MAX_OVERDUE",

  "CC_AMT_DRAWINGS_ATM_CURRENT_MIN",

  "CC_AMT_DRAWINGS_OTHER_CURRENT_MAX",

  "CC_AMT_DRAWINGS_OTHER_CURRENT_MIN",

  "CC_CNT_DRAWINGS_ATM_CURRENT_MIN",

  "CC_CNT_DRAWINGS_OTHER_CURRENT_MAX",

  "CC_CNT_DRAWINGS_OTHER_CURRENT_MIN",

  "CC_SK_DPD_DEF_MIN",

  "CC_SK_DPD_MIN",

  "BERB_STATUS_CREDIT_TYPE_Loan_for_working_capital_replenishment",

 "BERB_STATUS_CREDIT_TYPE_Real_estate_loan",

  "BERB_STATUS_CREDIT_TYPE_Loan_for_the_purchase_of_equipment",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Loan_for_working_capital_replenishmentClosed",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Car_loanSold",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Another_type_of_loanActive",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Loan_for_working_capital_replenishmentSold",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_MicroloanSold",

  "BERB_COMBO_CT_CA_COMBO_CT_CA_Another_type_of_loanSold",

  "FLAG_EMAIL",

  "APARTMENTS_AVG",

  "AMT_REQ_CREDIT_BUREAU_MON",

  "AMT_REQ_CREDIT_BUREAU_QRT",

  "AMT_REQ_CREDIT_BUREAU_YEAR",

  "STCK_BERBAL_6_",

  "STCK_CC_6_x"]

In [146]:
feats = [f for f in cols_to_drop if f in df.columns]

In [147]:
df.drop(labels=feats, axis=1, inplace=True)

In [148]:
cat_features = []

In [149]:
df.shape

(356255, 1160)

In [150]:
gc.collect()

7

In [151]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print('[{' + name + '}] done in {' + str(round(time.time() - t0, 3)) + '} s')
    

In [152]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras as ks
from keras import backend as K
import gc
from contextlib import contextmanager
import tensorflow as tf

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [153]:
encoding = 'ohe'

train_df = df.iloc[0:n_train]
test_df = df.iloc[n_train:]

print("Starting ANN. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
gc.collect()
# Cross validation model
folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

#feats = [col for col in feats_0 if df[col].dtype == 'object']


print(train_df[feats].shape)
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train['TARGET'])):
        
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
        
        if encoding == 'ohe':
            
            enc = ce.OneHotEncoder(impute_missing=True, cols=categorical_columns).fit(train_df[feats].iloc[train_idx],
                                                                                       train['TARGET'].iloc[train_idx])
            x_train = enc.transform(train_df[feats].iloc[train_idx]).replace([-np.inf, np.inf], np.nan).fillna(-999)
            x_valid = enc.transform(train_df[feats].iloc[valid_idx]).replace([-np.inf, np.inf], np.nan).fillna(-999)
            x_test = enc.transform(test_df[feats]).replace([-np.inf, np.inf], np.nan).fillna(-999)
            gc.collect()
            scaler = preprocessing.RobustScaler(quantile_range=(5.0, 95.0), with_scaling=True, with_centering=True)
            scaler.fit(x_train)
            x_train = scaler.transform(x_train)
            x_valid = scaler.transform(x_valid)
            x_test = scaler.transform(x_test)
            
            print(x_train.shape, x_valid.shape, x_test.shape)
        
        file_path = "best_model.hdf5"
        check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1,
                              save_best_only=True, mode="min")
        ra_val = RocAucEvaluation(validation_data=(x_valid, train['TARGET'].iloc[valid_idx].values), interval=1)
        early_stop = EarlyStopping(monitor="val_loss", mode = "min", patience=1)
        gc.collect()
        
        config = tf.ConfigProto(
        intra_op_parallelism_threads=6, use_per_session_threads=6, inter_op_parallelism_threads=6)
        with tf.Session(graph=tf.Graph(), config=config) as sess, timer('fit_predict'):
                    ks.backend.set_session(sess)
                    model_in = ks.Input(shape=(x_train.shape[1],), dtype='float32', sparse=False)
                    out = ks.layers.Dense(2 ** 11,  activation='sigmoid', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(model_in)
                    out = ks.layers.Dropout(0.5)(out)
                    out =  ks.layers.Dense(2 ** 9, activation='sigmoid', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(out)
                    out = ks.layers.Dropout(0.3)(out)
                    out =  ks.layers.Dense(2 ** 8, activation='relu', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(out)
                    out = ks.layers.Dropout(0.3)(out)
                    out = ks.layers.Dense(1, activation='sigmoid', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(out)
                    model = ks.models.Model(model_in, out)
                    model.compile(loss='binary_crossentropy',
                                  optimizer=ks.optimizers.Adam(lr=1e-3), metrics=['binary_crossentropy'])
                    batch_size = 2 ** 10
                    for i in range(3):
                        with timer('pass ' +  str(i + 1)):
                            model.fit(x=x_train, y=train['TARGET'].iloc[train_idx].values, batch_size=batch_size+(batch_size*(2*i)),
                                      epochs=20, validation_data=(x_valid, train['TARGET'].iloc[valid_idx].values), callbacks=[ra_val, 
                                      check_point, early_stop],
                                      shuffle=True)
                    y_pred = model.predict(x_valid).reshape(-1, 1)
                    print(roc_auc_score(y_true=train['TARGET'].iloc[valid_idx].values, y_score=y_pred))
                    gc.collect()   
                    oof_preds[valid_idx] = model.predict(x_valid)[:, 0]
                    sub_preds += model.predict(x_test)[:, 0] / folds.n_splits
                    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(train['TARGET'], oof_preds))
             


Starting ANN. Train shape: (307511, 1160), test shape: (48744, 1160)
(307511, 1158)
(246008, 1161) (61503, 1161) (48744, 1161)
Train on 246008 samples, validate on 61503 samples
Epoch 1/20
246008/246008 [==============================] - 65s 262us/step - loss: 0.2689 - binary_crossentropy: 0.2689 - val_loss: 0.2524 - val_binary_crossentropy: 0.2524

 ROC-AUC - epoch: 1 - score: 0.755923

Epoch 00001: val_loss improved from inf to 0.25244, saving model to best_model.hdf5
Epoch 2/20
246008/246008 [==============================] - 63s 255us/step - loss: 0.2492 - binary_crossentropy: 0.2492 - val_loss: 0.2495 - val_binary_crossentropy: 0.2495

 ROC-AUC - epoch: 2 - score: 0.762791

Epoch 00002: val_loss improved from 0.25244 to 0.24953, saving model to best_model.hdf5
Epoch 3/20
246008/246008 [==============================] - 63s 255us/step - loss: 0.2470 - binary_crossentropy: 0.2470 - val_loss: 0.2494 - val_binary_crossentropy: 0.2494

 ROC-AUC - epoch: 3 - score: 0.766710

Epoch 00003

KeyboardInterrupt: 

In [ ]:
roc_auc_score(train['TARGET'], oof_preds)

In [ ]:
print('Full AUC score %.6f' % roc_auc_score(train['TARGET'], oof_preds))
        
sub_df = test[['SK_ID_CURR']].copy()
sub_df['TARGET'] = sub_preds
sub_df[['SK_ID_CURR', 'TARGET']].to_csv(test_file_path, index= False)

val_df = train[['SK_ID_CURR', 'TARGET']].copy()
val_df['TARGET'] = oof_preds
val_df[['SK_ID_CURR', 'TARGET']].to_csv(validation_file_path, index= False)        
            
            
        
        
    

